# Sandbox for testing code
## Project: Algorithmic bicycle network design
#### Michael Szell, Tyler Perlman, Sayat Mimar, Gourab Ghoshal, Roberta Sinatra

## Preliminaries

### Parameters

In [ ]:
debug = False # If True, will produce plots and/or verbose output to double-check
%run -i "../parameters/parameters.py"

### Setup

In [ ]:
%run -i path.py
%run -i setup.py

%load_ext watermark
%watermark -n -v -m -g -iv

### Functions

In [ ]:
%run -i functions.py

## Delete plots

In [ ]:
for folder, subfolders, files in os.walk(PATH["plots"]):
    for file in files:
        if not file.endswith('_analysis_poi_grid_betweenness.png'):
            os.remove(os.path.join(folder, file))
            
for folder, subfolders, files in os.walk(PATH["plots_networks"]):
    for file in files:
        if file.endswith('_analysis_poi_grid_betweenness.png'):
            os.remove(os.path.join(folder, file))

## Graph overlap

In [ ]:
%run -i functions.py
%run -i "../parameters/parameters.py"
for placeid, placeinfo in cities.items():

    # Load networks
    G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall')
    Gs = {}
    for networktype in networktypes:
        Gs[networktype] = csv_to_ig(PATH["data"] + placeid + "/", placeid, networktype)   
    
    # Load POIs
    with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source + '_nnidscarall.csv') as f:
        nnids = [int(line.rstrip()) for line in f]
            
    # Load results
    filename = placeid + '_poi_' + poi_source + "_" + prune_measure
    resultfile = open(PATH["results"] + placeid + "/" + filename + ".pickle",'rb')
    res = pickle.load(resultfile)
    resultfile.close()
    
    
    # Test
    G1 = copy.deepcopy(Gs["biketrack"])
    G2 = copy.deepcopy(res["GTs"][-1])
    Ginter = intersect_igraphs(G1,G2)
    
    print(edge_lengths(G1), edge_lengths(G2), edge_lengths(Ginter))
    
    fig = plt.figure(figsize=(plotparam["bbox"][0]/plotparam["dpi"], plotparam["bbox"][1]/plotparam["dpi"]), dpi=plotparam["dpi"]); plt.axes().set_aspect('equal')
    map_center = nxdraw(G_carall, "carall")
    nxdraw(G1, "biketrack", map_center, list(set([v["id"] for v in G1.vs]).intersection(set([v["id"] for v in G_carall.vs]))))
    nxdraw(G2, "bikegrown_nonodes", map_center)
    nxdraw(Ginter, "highlight", map_center)
    

In [ ]:
Audio(sound_file, autoplay=True)

## Plot carminusbike network to see component growth (05_plot_results)

In [ ]:
%%capture
for placeid, placeinfo in cities.items():
    print(placeid + ": Plotting networks")
    
    # EXISTING INFRASTRUCTURE
    # Load networks
    G_biketrack = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'biketrack')
    G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall')
    G_biketrackcarall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'biketrackcarall')
    G_bikeable = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'bikeable')
    map_center = nxdraw(G_carall, "carall")

    # Load POIs
    with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source + '_nnidscarall.csv') as f:
        nnids = [int(line.rstrip()) for line in f]
    nodesize_poi = nodesize_from_pois(nnids)
    
    # Load results
    filename = placeid + '_poi_' + poi_source + "_" + prune_measure + ".pickle"
    with open(PATH["results"] + placeid + "/" + filename, 'rb') as f:
        res = pickle.load(f)
    if debug: pp.pprint(res)
        
    G_carminusbike = copy.deepcopy(G_carall)
    # PLOT all together (this can take some minutes)
    for GT, prune_quantile in zip(res["GTs"], res["prune_quantiles"]):
        delete_overlaps(G_carminusbike, GT)
        fig = initplot()
        nxdraw(G_carminusbike, "carall")
#         nxdraw(GT, "b'kegrown", map_center, nodesize = nodesize_grown)
        nxdraw(G_carall, "poi_unreached", map_center, nnids, "nx.draw_networkx_nodes", nodesize_poi)
        nxdraw(G_carall, "poi_reached", map_center, list(set([v["id"] for v in GT.vs]).intersection(set(nnids))), "nx.draw_networkx_nodes", nodesize_poi)
        plt.savefig(PATH["plots"] + placeid + "/" + placeid + '_carminusGT_poi_' + poi_source + "_" + prune_measures[prune_measure] + "{:.3f}".format(prune_quantile) + '.png', bbox_inches="tight", dpi=plotparam["dpi"])
        plt.close()